In [19]:
import pandas as pd
import numpy as np
import pymysql
import pandas.io.sql as psql
import matplotlib.pyplot as plt
import scipy.spatial as sp
import time



import sklearn 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import AdaBoostRegressor

from sklearn.metrics import confusion_matrix
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

c:\users\c25351\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\c25351\anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [50]:
con = pymysql.connect(
    host= ********
    user= ********,
    password=********,
    db='checkouts_by_title',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)
con

In [51]:
sql = """
SELECT
log.title,
log.checkout_year,
log.checkout_month,
sum(log.checkouts)
FROM checkouts_by_title.checkouts_log_sample as log
WHERE checkout_year > 2013
GROUP BY title, checkout_year, checkout_month
"""

In [74]:
data = psql.read_sql(con=con, sql=sql)

In [75]:
data.head()

,title,checkout_year,checkout_month,sum(log.checkouts)
0,"""A"" is for alibi [text (large print)] / Sue Gr...",2014,8,1.0
1,"""Come Out of Your Shell"" & ""The Big Picture""",2015,4,1.0
2,"""D"" is for Deadbeat: Kinsey Millhone Series, B...",2015,4,3.0
3,"""H"" is for homicide [sound recording] / by Sue...",2014,10,2.0
4,"""I"" is for innocent [sound recording] / by Sue...",2015,11,2.0


In [76]:
dataF = data

In [77]:
dataF.shape

(21834, 4)

In [82]:
#dataF["YM"] = pd.to_datetime(data["checkout_year"].dt.datetime("") + data["checkout_month"])
dataF["YM"] = data["checkout_year"] + data["checkout_month"]

In [66]:
dataF.drop(["checkout_year","checkout_month"],axis=1, inplace=True)

In [68]:
dataF["sum"] = dataF["sum(log.checkouts)"]
dataF.drop(["sum(log.checkouts)"],axis=1, inplace=True)

In [69]:
dataF.head

<bound method NDFrame.head of                                                    title      YM   sum
0      "A" is for alibi [text (large print)] / Sue Gr...   20148   1.0
1           "Come Out of Your Shell" & "The Big Picture"   20154   1.0
2      "D" is for Deadbeat: Kinsey Millhone Series, B...   20154   3.0
3      "H" is for homicide [sound recording] / by Sue...  201410   2.0
4      "I" is for innocent [sound recording] / by Sue...  201511   2.0
5      "I'm rich beyond your wildest dreams" [sound r...   20147   3.0
6            "Irish" all time hits [music] : song album.   20153   1.0
7      "Let's talk about sex" [videorecording] : a re...   20147   5.0
8         "Master Harold"-- and the boys / Athol Fugard.  201510   1.0
9      "Only a poor old man" / Carl Barks ; [edited b...   20146   8.0
10     "Scribbling women" : true tales from astonishi...   20158   3.0
11     "W" is for Wasted: Kinsey Millhone Series, Boo...   20145  35.0
12     "Who could that be at this hour?" [text 

In [70]:
#title × YM
dataF_pivot = pd.pivot_table(dataF, index = "title", columns= "YM", values = "sum" ).fillna(0)

In [139]:
dataF_pivot["5m_train"] = dataF_pivot.loc[:,"20152":"20156"].sum(axis = 1)
dataF_pivot["5mmax_train"] = dataF_pivot.loc[:,"20152":"20156"].max(axis = 1)

dataF_pivot["5m_test"] = dataF_pivot.loc[:,"20153":"20157"].sum(axis = 1)
dataF_pivot["5mmax_test"] = dataF_pivot.loc[:,"20153":"20157"].max(axis = 1)

dataF_pivot["1Y_train"] = dataF_pivot.loc[:,"20146":"20156"].sum(axis = 1)
dataF_pivot["1Ymax_train"] = dataF_pivot.loc[:,"20146":"20156"].max(axis = 1)

dataF_pivot["1Y_test"] = dataF_pivot.loc[:,"20147":"20157"].sum(axis = 1)
dataF_pivot["1Ymax_test"] = dataF_pivot.loc[:,"20147":"20157"].max(axis = 1)

dataF_pivot["train_test"] = dataF_pivot["20157"]
dataF_pivot["target_test"] = dataF_pivot["20158"]

dataF_pivot.head()

YM,20141,201410,201411,201412,20142,20143,20144,20145,20146,20147,...,3m_train,3m_test,1Y_train,1Y_test,train_test,target_test,5m_train,5m_test,5mmax_train,5mmax_test
title,,,,,,,,,,,,,,,,,,,,,
"""A"" is for alibi [text (large print)] / Sue Grafton.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Come Out of Your Shell"" & ""The Big Picture""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
"""D"" is for Deadbeat: Kinsey Millhone Series, Book 4 (unabridged) (Unabridged)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,3.0,0.0,0.0,3.0,3.0,3.0,3.0
"""H"" is for homicide [sound recording] / by Sue Grafton.",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""I"" is for innocent [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
train = dataF_pivot.drop(dataF_pivot.loc[:,:"target_test"], axis= 1)
test = train
train.head()

YM,5m_train,5m_test,5mmax_train,5mmax_test
title,,,,
"""A"" is for alibi [text (large print)] / Sue Grafton.",0.0,0.0,0.0,0.0
"""Come Out of Your Shell"" & ""The Big Picture""",1.0,1.0,1.0,1.0
"""D"" is for Deadbeat: Kinsey Millhone Series, Book 4 (unabridged) (Unabridged)",3.0,3.0,3.0,3.0
"""H"" is for homicide [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0
"""I"" is for innocent [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0


In [142]:
train["5m"] = dataF_pivot["5m_train"] 
train["1Y"] = dataF_pivot["1Y_train"]
train["B1"] = dataF_pivot["20156"]
train["B2"] = dataF_pivot["20155"]
train["B3"] = dataF_pivot["20154"]
train["5mmax"] = dataF_pivot["5mmax_train"] 
train["1Ymax"] = dataF_pivot["1Ymax_train"] 

test["5m"] = dataF_pivot["5m_test"] 
test["1Y"] = dataF_pivot["1Y_test"]
test["B1"] = dataF_pivot["20157"]
test["B2"] = dataF_pivot["20156"]
test["B3"] = dataF_pivot["20155"]
test["5mmax"] = dataF_pivot["5mmax_test"] 
test["1Ymax"] = dataF_pivot["1Ymax_test"] 


train["target"] = dataF_pivot["20157"]
test["target"] = dataF_pivot["20158"]

train["target"] = train["target"].apply(lambda x : 1 if x > 0 else 0)
test["target"] = test["target"].apply(lambda x : 1 if x > 0 else 0)

In [143]:
train.head()

YM,5m_train,5m_test,5mmax_train,5mmax_test,5m,1Y,B1,B2,B3,5mmax,target,1Ymax
title,,,,,,,,,,,,
"""A"" is for alibi [text (large print)] / Sue Grafton.",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1.0
"""Come Out of Your Shell"" & ""The Big Picture""",1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0,1.0
"""D"" is for Deadbeat: Kinsey Millhone Series, Book 4 (unabridged) (Unabridged)",3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,3.0,0,3.0
"""H"" is for homicide [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
"""I"" is for innocent [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0,2.0


In [144]:
test.head()

YM,5m_train,5m_test,5mmax_train,5mmax_test,5m,1Y,B1,B2,B3,5mmax,target,1Ymax
title,,,,,,,,,,,,
"""A"" is for alibi [text (large print)] / Sue Grafton.",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1.0
"""Come Out of Your Shell"" & ""The Big Picture""",1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0,1.0
"""D"" is for Deadbeat: Kinsey Millhone Series, Book 4 (unabridged) (Unabridged)",3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,3.0,0,3.0
"""H"" is for homicide [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
"""I"" is for innocent [sound recording] / by Sue Grafton.",0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0,2.0


In [145]:
target_col = "target"
exclude_cols = ["target"]
feature_cols = [col for col in train.columns if col not in exclude_cols]

In [146]:
y = np.array(train[target_col])
X = np.array(train[feature_cols])

In [147]:
X_train1, X_train2, y_train1, y_train2 = \
 train_test_split(X, y, test_size=0.3, random_state=1234)

In [148]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

In [149]:
################当日の1/0をロジスティック回帰する########
#ロジスティック回帰の定義
clf_lr = LogisticRegression(n_jobs=-1, class_weight= "balanced")
clf_lr.fit(X_train1, y_train1)
y_test_pred_lo_tr = clf_lr.predict(X_train2)
show_evaluation_metrics(y_train2, y_test_pred_lo_tr)

Accuracy:
0.549645948072384

Report:
             precision    recall  f1-score   support

          0       1.00      0.53      0.69      6073
          1       0.09      0.98      0.16       282

avg / total       0.96      0.55      0.67      6355

Confusion matrix:
[[3217 2856]
 [   6  276]]


c:\users\c25351\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [150]:
### SVM (線形カーネル)
clf_svc = LinearSVC(class_weight= "balanced")
clf_svc.fit(X_train1, y_train1)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [151]:
y_test_pred = clf_svc.predict(X_train2)
show_evaluation_metrics(y_train2, y_test_pred)

Accuracy:
0.5751376868607396

Report:
             precision    recall  f1-score   support

          0       1.00      0.56      0.71      6073
          1       0.09      0.98      0.17       282

avg / total       0.96      0.58      0.69      6355

Confusion matrix:
[[3380 2693]
 [   7  275]]


In [163]:
### Random Forest
clf_rf = RandomForestClassifier(n_estimators=10, n_jobs=-1,class_weight= "balanced")
#clf_rf = RandomForestClassifier(class_weight= "balanced")

clf_rf.fit(X_train1, y_train1)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [164]:
y_test_pred = clf_rf.predict(X_train2)
show_evaluation_metrics(y_train2, y_test_pred)

Accuracy:
0.597639653815893

Report:
             precision    recall  f1-score   support

          0       1.00      0.58      0.73      6073
          1       0.10      0.97      0.18       282

avg / total       0.96      0.60      0.71      6355

Confusion matrix:
[[3524 2549]
 [   8  274]]


In [154]:
#####testに適用

In [155]:
target_col = "target"
exclude_cols = ["target"]
feature_cols = [col for col in test.columns if col not in exclude_cols]

In [156]:
y = np.array(test[target_col])
X = np.array(test[feature_cols])

In [157]:
y_test_pred_lo_te = clf_lr.predict(X)
show_evaluation_metrics(y, y_test_pred_lo_te)

Accuracy:
0.5409309791332263

Report:
             precision    recall  f1-score   support

          0       1.00      0.52      0.68     20211
          1       0.09      0.97      0.16       971

avg / total       0.96      0.54      0.66     21182

Confusion matrix:
[[10515  9696]
 [   28   943]]


In [158]:
### SVM (線形カーネル)
y_test_pred = clf_svc.predict(X)
show_evaluation_metrics(y, y_test_pred)

Accuracy:
0.5690680766688698

Report:
             precision    recall  f1-score   support

          0       1.00      0.55      0.71     20211
          1       0.09      0.97      0.17       971

avg / total       0.96      0.57      0.68     21182

Confusion matrix:
[[11113  9098]
 [   30   941]]


In [159]:
### Random Forest
y_test_pred = clf_rf.predict(X)
show_evaluation_metrics(y, y_test_pred)

Accuracy:
0.5905957888773486

Report:
             precision    recall  f1-score   support

          0       1.00      0.57      0.73     20211
          1       0.10      0.97      0.18       971

avg / total       0.96      0.59      0.70     21182

Confusion matrix:
[[11567  8644]
 [   28   943]]
